In [95]:
import pandas as pd
import numpy as np

df = pd.read_csv('pre_processed.csv')


# Add a column for potential decks

In [96]:
homeplanets = ['Earth', 'Europa', 'Mars']

In [97]:
decks_by_planet = {
    'Earth':['E','F','G'],
    'Europa': ['A','B','C','D','E','T'],
    'Mars': ['D','E','F']
}

decks_by_planet_no_bills = {
    'Earth':['G'],
    'Europa':['B'],
    'Mars': ['E','F']
}

planet_by_deck = {
    'A':['Europa'],'B':['Europa'],'C':['Europa'],'D':['Europa','Mars'],'E':['Europa','Mars','Earth'],
    'F':['Earth','Mars'],'G':['Earth'],'T':['Europa']
}

cabin_sides = ['P','S']

cabin_decks = list(df.dropna(subset = ['Deck']).Deck.unique())


In [98]:
def deck_options(df,row):
    if pd.isna(row.HomePlanet):
        return 'no homeplanet' 
    elif row.Bills == 0:
        if len(df[df.Group == row.Group].dropna(subset = 'Deck').Deck.unique()) > 1:
            return decks_by_planet_no_bills[row.HomePlanet]
    return decks_by_planet[row.HomePlanet]

In [99]:
def multiple_decks_in_group(df,row):
    return len(df[df.Group == row.Group].dropna(subset ='Deck').Deck.unique()) > 1
        

# Filling missed Cabins


In [100]:
def utilised_cabins(df):
    used_cabins = {}
    for deck in list(df.dropna().Deck.unique()):
        cab = {}
        for side in list(df.dropna().Side.unique()):
            rooms = list(df[(df.Deck == deck) & (df.Side == side)].dropna().Number.unique())
            rooms.sort()
            cab[side] = rooms
        used_cabins[deck] = cab
    return used_cabins
    

#### fill people that must be sharing a room with people in the same group
ie, from their chome planet we know the decks they could be in, if the side and number below and above are one above and one below then they must be sharing with someone from their group. make sure that their group has only one cabin

need to make sure is only one person  with cabin being na in the group otherwise one na might be in another cabin 

for row in dataframe

if cabin.isna() and homeplanet is known,
 for potential decks in homeplanets remit ie europa check a, b , c in dataframe up to that point and over that point, see if any have a free space and if not then fill it with the one in the same group

In [101]:
def checks(df):
    for home in homeplanets:
        print("\n" + home)
        print(df[df.HomePlanet == home].Deck.value_counts())
    return df

In [102]:
df2 = df.copy()

Finding groups that have more than 1 member that all share the same cabin and that dont have other nans in group

In [144]:
df2.isna().sum()

PassengerId        0
HomePlanet        13
CryoSleep        310
Cabin             76
Destination      274
Age              270
VIP              296
RoomService      263
FoodCourt        289
ShoppingMall     306
Spa              284
VRDeck           268
Name             294
Set                0
Transported     4277
Group              0
GroupNumber        0
Deck             145
Number           145
Side             145
FristName        294
LastName         294
GroupSize          0
Bills            785
dtype: int64

# one room available and its alone in its group ( should be first in cabin fill)

In [145]:
def one_room_available_and_alone_in_group(df):
    for index, passenger in df[df.Cabin.isna()].iterrows():
        options = []
        for deck in decks_by_planet[passenger.HomePlanet]:
            for side in cabin_sides:
                
                before_slice = df.iloc[:index]
                after_slice = df.iloc[index+1:] 
                top_room_number_before = np.max(before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number)
                smallest_room_number_after = np.min(after_slice[(after_slice.Deck == deck) & (after_slice.Side == side)].Number)
                
                if top_room_number_before + 1 != smallest_room_number_after:
                    if pd.isna(top_room_number_before) and smallest_room_number_after == 0:
                        continue
                    elif top_room_number_before == smallest_room_number_after:
                        continue
                    else:
                        options.append([deck,side,top_room_number_before,smallest_room_number_after])
        
        if len(options) == 1:
            option = options[0]
            if option[2] + 2 == option[3] and len(df[df.Group == passenger.Group]) == 1:
                df.loc[index,['Side','Deck','Number']] = [option[1],option[0],int(option[2] + 1)]
                df.loc[index,'Cabin'] = str(option[0]) + "/" + str(int(option[2] + 1)) + "/" + str(option[1])
    return df

df2 = one_room_available_and_alone_in_group(df2)

                
    
    

    

In [146]:
df2.isna().sum()

PassengerId        0
HomePlanet        13
CryoSleep        310
Cabin             76
Destination      274
Age              270
VIP              296
RoomService      263
FoodCourt        289
ShoppingMall     306
Spa              284
VRDeck           268
Name             294
Set                0
Transported     4277
Group              0
GroupNumber        0
Deck             145
Number           145
Side             145
FristName        294
LastName         294
GroupSize          0
Bills            785
dtype: int64

# there arent any free rooms for it so has to share


In [147]:
def no_free_rooms_so_shares(df):
    for index, passenger in df[df.Cabin.isna()].iterrows():
        options = False
        for deck in decks_by_planet[passenger.HomePlanet]:
            for side in cabin_sides:
                
                before_slice = df.iloc[:index]
                after_slice = df.iloc[index+1:] 
                top_room_number_before = np.max(before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number)
                smallest_room_number_after = np.min(after_slice[(after_slice.Deck == deck) & (after_slice.Side == side)].Number)
                
                if top_room_number_before + 1 != smallest_room_number_after:
                    if pd.isna(top_room_number_before) and smallest_room_number_after == 0:
                        continue
                    elif top_room_number_before == smallest_room_number_after:
                        continue
                    else:
                        options = True
                        break
            if options:
                break
        if not options:
            other_group_member = df[(df.Group == passenger.Group) & (~df.Cabin.isna())]
            if len(other_group_member.Cabin.unique()) == 1:
                df.loc[index,'Cabin'] = other_group_member.iloc[0].Cabin
    return df
  
            # Slice the DataFrame first and then apply the boolean mask

df2 = no_free_rooms_so_shares(df2)

In [149]:
df2.isna().sum()

PassengerId        0
HomePlanet        13
CryoSleep        310
Cabin             76
Destination      274
Age              270
VIP              296
RoomService      263
FoodCourt        289
ShoppingMall     306
Spa              284
VRDeck           268
Name             294
Set                0
Transported     4277
Group              0
GroupNumber        0
Deck             145
Number           145
Side             145
FristName        294
LastName         294
GroupSize          0
Bills            785
dtype: int64

Cases
* there arent any free rooms for it so has to share
* its the only person that could fill that room 

# free room where only one person can take it

In [150]:
def rooms_to_fill(df):
    rooms = {}
    for deck in cabin_decks:
        for side in cabin_sides:
            rooms_seen = list(df[(df.Deck == deck) & (df.Side == side)].dropna(subset = ['Number']).Number.unique())
            largest_room_number = int(max(list(df[(df.Deck == deck) & (df.Side == side)].dropna(subset = ['Number']).Number.unique())))
            for i in range(largest_room_number):
                if i not in rooms_seen:
                    if deck not in rooms:
                        rooms[deck] = {'P':[],'S':[]}
                    rooms[deck][side].append(i)
    return rooms

rooms = rooms_to_fill(df2)
                    
                    
                    
            

In [151]:
def fill_by_empty_room(df):
    for deck in rooms.keys():
        for side in ['P','S']:
            for number in rooms[deck][side]:
                indices_matching = []
                temp = df[(df.Cabin.isna()) & (pd.isna(df.HomePlanet) | df.HomePlanet.isin(planet_by_deck[deck]))]
                for index,row in temp.iterrows():
                    if row.Bills == 0 and not pd.isna(row.HomePlanet):
                        if multiple_decks_in_group(df,row):
                            if deck not in decks_by_planet_no_bills[row.HomePlanet]:
                                continue
                            
                    before_slice = df.iloc[:index]
                    after_slice = df.iloc[index+1:] 
                    if max(list(before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number.unique())) < number:
                        if min(list(after_slice[(after_slice.Deck == deck) & (after_slice.Side == side)].Number.unique())) > number:
                            indices_matching.append(index)
                if len(indices_matching) == 1:
                    df.loc[indices_matching[0],'Cabin'] = deck + "/" + str(number) + "/" + side
                    df.loc[indices_matching[0],['Side','Deck','Number']] = [side,deck,number]
fill_by_empty_room(df2)

In [153]:
df2[df2.Cabin.isna()]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Transported,Group,GroupNumber,Deck,Number,Side,FristName,LastName,GroupSize,Bills
404,0293_01,Europa,True,NaN,TRAPPIST-1e,47.0,False,0.0,0.0,0.0,...,NaN,293,1,NaN,NaN,NaN,Tauxon,Suptibler,1,0.0
421,0310_01,Europa,False,NaN,TRAPPIST-1e,67.0,False,NaN,230.0,0.0,...,False,310,1,NaN,NaN,NaN,Naviton,Coudered,1,NaN
479,0348_02,Mars,NaN,NaN,TRAPPIST-1e,36.0,False,520.0,0.0,1865.0,...,True,348,2,NaN,NaN,NaN,Weet,Mane,2,2385.0
505,0364_02,Mars,False,NaN,TRAPPIST-1e,37.0,False,731.0,0.0,517.0,...,NaN,364,2,NaN,NaN,NaN,Anakes,Chité,2,1298.0
517,0374_02,Earth,False,NaN,TRAPPIST-1e,36.0,False,6.0,0.0,0.0,...,NaN,374,2,NaN,NaN,NaN,Tamie,Sterreray,2,789.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12658,9062_02,Earth,True,NaN,TRAPPIST-1e,65.0,False,0.0,0.0,0.0,...,NaN,9062,2,NaN,NaN,NaN,Lerion,Pughlinsons,2,0.0
12668,9069_03,Europa,True,NaN,55 Cancri e,25.0,False,0.0,0.0,0.0,...,True,9069,3,NaN,NaN,NaN,Bath,Brakeng,5,0.0
12671,9070_01,Earth,False,NaN,TRAPPIST-1e,20.0,False,323.0,0.0,542.0,...,NaN,9070,1,NaN,NaN,NaN,Oraryn,Flynney,2,865.0
12892,9223_01,Mars,True,NaN,TRAPPIST-1e,24.0,False,0.0,0.0,NaN,...,NaN,9223,1,NaN,NaN,NaN,Weessh,Sun,2,0.0


In [154]:
df3 = pd.read_csv('recentattempts.csv')

In [155]:
df3.isna().sum()

PassengerId        0
HomePlanet        13
CryoSleep        310
Cabin             89
Destination      274
Age              270
VIP              296
RoomService      263
FoodCourt        289
ShoppingMall     306
Spa              284
VRDeck           268
Name             294
Set                0
Transported     4277
Group              0
GroupNumber        0
Deck             145
Number           145
Side             145
FristName        294
LastName         294
GroupSize          0
Bills            785
dtype: int64

# remaining cabins

# which cabin for empty people

In [182]:
def remaining_cabins(df):
    for index, passenger in df[df.Cabin.isna()].iterrows():
        print("\nindex", index)
        print("passenger",passenger.PassengerId)
        options = []
        for deck in decks_by_planet[passenger.HomePlanet]:
            for side in cabin_sides:
                
                before_slice = df.iloc[:index]
                after_slice = df.iloc[index+1:] 
                top_room_number_before = np.max(before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number)
                smallest_room_number_after = np.min(after_slice[(after_slice.Deck == deck) & (after_slice.Side == side)].Number)
                
                if top_room_number_before + 1 != smallest_room_number_after:
                    if pd.isna(top_room_number_before) and smallest_room_number_after == 0:
                        continue
                    elif top_room_number_before == smallest_room_number_after:
                        continue
                    else:
                        options.append([deck,side,top_room_number_before,smallest_room_number_after])
        print(options)


# which people for empty cabin

In [183]:
def fill_by_empty_room(df):
    for deck in rooms.keys():
        for side in ['P','S']:
            for number in rooms[deck][side]:
                indices_matching = []
                print()
                print('deck',deck,'side',side,'number',number)
                temp = df[(df.Cabin.isna()) & (pd.isna(df.HomePlanet) | df.HomePlanet.isin(planet_by_deck[deck]))]
                for index,row in temp.iterrows():
                    before_slice = df.iloc[:index]
                    after_slice = df.iloc[index+1:] 
                    if max(list(before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number.unique())) < number:
                        if min(list(after_slice[(after_slice.Deck == deck) & (after_slice.Side == side)].Number.unique())) > number:
                            indices_matching.append(index)
                print(df.iloc[indices_matching].PassengerId)
                print()
                print()


## what can be deduced with these cabins now filled ?

# no free rooms so has to share but with no bills

In [176]:
def no_free_rooms_no_bills(df):
    for index, passenger in df[df.Cabin.isna()].iterrows():
        options = False
        print()
        if passenger.Bills == 0 and not pd.isna(passenger.HomePlanet):
            if multiple_decks_in_group(df,passenger):
                print(index)
                for deck in decks_by_planet_no_bills[passenger.HomePlanet]:
                    for side in cabin_sides:
                        
                        before_slice = df.iloc[:index]
                        after_slice = df.iloc[index+1:] 
                        top_room_number_before = np.max(before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number)
                        smallest_room_number_after = np.min(after_slice[(after_slice.Deck == deck) & (after_slice.Side == side)].Number)
                        
                        if top_room_number_before + 1 != smallest_room_number_after:
                            if pd.isna(top_room_number_before) and smallest_room_number_after == 0:
                                continue
                            elif top_room_number_before == smallest_room_number_after:
                                continue
                            else:
                                options = True
                                break
                    if options:
                        break
                if not options:
                    if len(df[df.Group == passenger.Group].Cabin) - df[df.Group == passenger.Group].Cabin.isna().sum() > 1:
                        other_group_member = df[(df.Group == passenger.Group) & (~df.Cabin.isna())]
                        options = deck_options(df,passenger)
                        if len(options) == 1:
                            if len(other_group_member[other_group_member.Deck == options[0]].Cabin.unique()) == 1:
                                df.loc[index,'Cabin'] = other_group_member[other_group_member.Deck == options[0]].iloc[0].Cabin
                        else:
                            print('multiple options for deck')
    return df
            
        
df2 = no_free_rooms_no_bills(df2)

In [205]:
df2[df2.Group == 9069]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Transported,Group,GroupNumber,Deck,Number,Side,FristName,LastName,GroupSize,Bills
12666,9069_01,Europa,True,B/297/P,TRAPPIST-1e,22.0,False,0.0,0.0,0.0,...,True,9069,1,B,297.0,P,Azhat,Brakeng,5,0.0
12667,9069_02,Europa,True,B/297/P,55 Cancri e,26.0,False,0.0,0.0,0.0,...,True,9069,2,B,297.0,P,Hekab,Brakeng,5,0.0
12668,9069_03,Europa,True,NaN,55 Cancri e,25.0,False,0.0,0.0,0.0,...,True,9069,3,NaN,NaN,NaN,Bath,Brakeng,5,0.0
12669,9069_04,Europa,False,B/297/P,TRAPPIST-1e,44.0,False,0.0,4313.0,0.0,...,True,9069,4,B,297.0,P,Cheleon,Brakeng,5,4888.0
12670,9069_05,Europa,False,B/297/P,55 Cancri e,29.0,False,0.0,12563.0,0.0,...,False,9069,5,B,297.0,P,Mirfark,Taketiatim,5,17623.0


In [194]:
df2[(df2.Cabin.isna()) & (df2.Bills == 0) & (df2.GroupSize > 2)]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Transported,Group,GroupNumber,Deck,Number,Side,FristName,LastName,GroupSize,Bills
1423,1006_03,Earth,False,NaN,TRAPPIST-1e,12.0,False,0.0,0.0,0.0,...,True,1006,3,NaN,NaN,NaN,Stenny,Summington,3,0.0
5857,4181_03,Europa,True,NaN,TRAPPIST-1e,17.0,False,0.0,0.0,0.0,...,True,4181,3,NaN,NaN,NaN,Terope,Frattyring,6,0.0
6709,4782_01,Europa,True,NaN,55 Cancri e,23.0,False,0.0,0.0,0.0,...,True,4782,1,NaN,NaN,NaN,Sabiton,Maxled,5,0.0
7333,5254_03,Europa,True,NaN,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,...,True,5254,3,NaN,NaN,NaN,Sargah,Pressive,4,0.0
8534,6111_03,Europa,True,NaN,55 Cancri e,21.0,False,0.0,0.0,0.0,...,True,6111,3,NaN,NaN,NaN,Ainkard,Folumorly,5,0.0
9267,6612_05,Earth,False,NaN,TRAPPIST-1e,2.0,False,0.0,0.0,0.0,...,NaN,6612,5,NaN,NaN,NaN,Maurie,Dickley,6,0.0
9273,6617_01,Europa,True,NaN,TRAPPIST-1e,31.0,False,0.0,0.0,0.0,...,NaN,6617,1,NaN,NaN,NaN,Mareson,Timpeed,3,0.0
11999,8605_02,Europa,True,NaN,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,...,True,8605,2,NaN,NaN,NaN,Mera,Netshaless,5,0.0
12668,9069_03,Europa,True,NaN,55 Cancri e,25.0,False,0.0,0.0,0.0,...,True,9069,3,NaN,NaN,NaN,Bath,Brakeng,5,0.0


In [212]:
for index,row in df2[df2.Cabin.isna()].iterrows():
    if multiple_decks_in_group(df2,row):
        if row.Bills == 0:
            print(index)

In [188]:
df2[df2.Group == df2.iloc[1401].Group]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Transported,Group,GroupNumber,Deck,Number,Side,FristName,LastName,GroupSize,Bills
1398,0992_01,Europa,False,D/35/S,TRAPPIST-1e,49.0,False,31.0,2727.0,0.0,...,True,992,1,D,35.0,S,Zino,Kiling,6,3077.0
1399,0992_02,Europa,False,D/35/S,TRAPPIST-1e,29.0,False,387.0,942.0,0.0,...,False,992,2,D,35.0,S,Tareson,Kiling,6,12597.0
1400,0992_03,Europa,False,D/35/S,55 Cancri e,46.0,False,1192.0,1736.0,17.0,...,True,992,3,D,35.0,S,Matan,Herpumble,6,2966.0
1401,0992_04,Europa,False,NaN,55 Cancri e,35.0,True,8030.0,300.0,0.0,...,False,992,4,NaN,NaN,NaN,Naosa,Herpumble,6,9766.0
1402,0992_05,Europa,True,D/35/S,55 Cancri e,17.0,False,0.0,0.0,0.0,...,True,992,5,D,35.0,S,Sinon,Kiling,6,0.0
1403,0992_06,Europa,False,D/35/S,55 Cancri e,37.0,False,4301.0,791.0,0.0,...,False,992,6,D,35.0,S,Weidus,Kiling,6,6931.0


In [184]:
remaining_cabins(df2)


index 404
passenger 0293_01
[['B', 'P', 12.0, 14.0], ['C', 'S', 12.0, 14.0]]

index 421
passenger 0310_01
[['B', 'P', 12.0, 14.0], ['C', 'S', 12.0, 14.0]]

index 479
passenger 0348_02
[['E', 'P', 19.0, 22.0]]

index 505
passenger 0364_02
[['E', 'P', 19.0, 22.0]]

index 517
passenger 0374_02
[['E', 'P', 19.0, 22.0]]

index 1401
passenger 0992_04
[['E', 'P', 57.0, 59.0]]

index 1423
passenger 1006_03
[['E', 'P', 57.0, 59.0]]

index 1429
passenger 1011_01
[['E', 'P', 57.0, 59.0]]

index 1466
passenger 1041_01
[['C', 'S', 39.0, 41.0], ['D', 'S', 35.0, 37.0], ['E', 'P', 57.0, 59.0]]

index 1543
passenger 1095_01
[['C', 'S', 39.0, 41.0], ['D', 'S', 35.0, 37.0]]

index 2442
passenger 1709_03
[]

index 2970
passenger 2092_03
[]

index 3529
passenger 2513_01
[['E', 'P', 149.0, 151.0], ['F', 'P', 518.0, 520.0]]

index 3530
passenger 2514_01
[['E', 'P', 149.0, 151.0], ['F', 'P', 518.0, 520.0]]

index 4233
passenger 3034_01
[['B', 'P', 97.0, 100.0]]

index 4254
passenger 3053_01
[['B', 'P', 97.0,

In [210]:
df2[df2.Group == 1709]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Transported,Group,GroupNumber,Deck,Number,Side,FristName,LastName,GroupSize,Bills
2440,1709_01,Mars,False,F/326/S,TRAPPIST-1e,23.0,False,31.0,0.0,1247.0,...,False,1709,1,F,326.0,S,Jurbet,Minen,7,1325.0
2441,1709_02,Mars,False,D/61/S,TRAPPIST-1e,23.0,False,2210.0,0.0,89.0,...,True,1709,2,D,61.0,S,Bet,Minen,7,2299.0
2442,1709_03,Mars,False,NaN,TRAPPIST-1e,35.0,False,1313.0,0.0,24.0,...,False,1709,3,NaN,NaN,NaN,Bleark,Minen,7,1338.0
2443,1709_04,Mars,False,E/127/S,TRAPPIST-1e,17.0,False,1744.0,0.0,136.0,...,False,1709,4,E,127.0,S,Donut,Minen,7,1896.0
2444,1709_05,Mars,True,F/326/S,TRAPPIST-1e,26.0,False,0.0,0.0,0.0,...,True,1709,5,F,326.0,S,Fams,Minen,7,0.0
2445,1709_06,Mars,False,D/61/S,TRAPPIST-1e,15.0,False,1554.0,0.0,646.0,...,False,1709,6,D,61.0,S,Tyog,Minen,7,3461.0
2446,1709_07,Mars,False,F/326/S,TRAPPIST-1e,19.0,False,285.0,54.0,1246.0,...,False,1709,7,F,326.0,S,Cocowl,Minen,7,1687.0


In [206]:
fill_by_empty_room(df2)



deck B side P number 13
404    0293_01
421    0310_01
Name: PassengerId, dtype: object



deck B side P number 98
4233    3034_01
4254    3053_01
Name: PassengerId, dtype: object



deck B side P number 99
4233    3034_01
4254    3053_01
Name: PassengerId, dtype: object



deck F side P number 519
3529    2513_01
3530    2514_01
Name: PassengerId, dtype: object



deck F side P number 1489
10081    7182_01
10082    7183_01
Name: PassengerId, dtype: object



deck F side P number 1544
10434    7463_01
10440    7469_01
Name: PassengerId, dtype: object



deck F side S number 1267
9265    6612_03
9267    6612_05
Name: PassengerId, dtype: object



deck F side S number 1424
10394    7429_01
10408    7440_01
10411    7442_02
10434    7463_01
Name: PassengerId, dtype: object



deck F side S number 1785
12892    9223_01
12893    9223_02
Name: PassengerId, dtype: object



deck A side P number 94
12651    9057_01
12668    9069_03
Name: PassengerId, dtype: object



deck G side P number 590
5

In [208]:
df2[(df2.GroupSize == 1) & (df2.Cabin.isna())]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Transported,Group,GroupNumber,Deck,Number,Side,FristName,LastName,GroupSize,Bills
404,0293_01,Europa,True,NaN,TRAPPIST-1e,47.0,False,0.0,0.0,0.0,...,NaN,293,1,NaN,NaN,NaN,Tauxon,Suptibler,1,0.0
421,0310_01,Europa,False,NaN,TRAPPIST-1e,67.0,False,NaN,230.0,0.0,...,False,310,1,NaN,NaN,NaN,Naviton,Coudered,1,NaN
1466,1041_01,Europa,True,NaN,TRAPPIST-1e,46.0,False,0.0,0.0,0.0,...,True,1041,1,NaN,NaN,NaN,Algrafi,Heedry,1,0.0
1543,1095_01,Europa,True,NaN,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,...,True,1095,1,NaN,NaN,NaN,Alhail,Drelcate,1,0.0
3529,2513_01,Earth,False,NaN,TRAPPIST-1e,28.0,False,0.0,55.0,0.0,...,False,2513,1,NaN,NaN,NaN,Loree,Mathison,1,711.0
3530,2514_01,Earth,False,NaN,TRAPPIST-1e,36.0,False,3.0,755.0,0.0,...,NaN,2514,1,NaN,NaN,NaN,Ianne,Doughan,1,791.0
4233,3034_01,Europa,False,NaN,TRAPPIST-1e,40.0,False,0.0,0.0,0.0,...,False,3034,1,NaN,NaN,NaN,Dscheat,Noxnuther,1,0.0
4254,3053_01,Europa,False,NaN,55 Cancri e,52.0,False,0.0,2.0,0.0,...,False,3053,1,NaN,NaN,NaN,Ainor,Fuelisent,1,11788.0
5016,3598_01,Earth,False,NaN,55 Cancri e,19.0,False,616.0,0.0,46.0,...,False,3598,1,NaN,NaN,NaN,Dony,Olivasquez,1,676.0
5017,3599_01,Earth,False,NaN,TRAPPIST-1e,46.0,False,48.0,856.0,0.0,...,NaN,3599,1,NaN,NaN,NaN,Philda,Newtontoss,1,908.0
